import pandas as pd
import numpy as np


df = pd.read_csv("competition_dataset/dev.tsv", sep="\t")

(df[pd.isna(df["region_1"]) & pd.isna(df["region_2"])])

df.info()

df

#with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
print(df["designation"].value_counts())

with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(df["designation"].value_counts()[:100])

#df[df["designation_tr"] == "reserva" | df["designation_tr"] == "riserva" | df["designation_tr"] == "rèserve"]["designation_tr"] = "reserve"
df["designation_tr"] = df["designation"].apply(lambda d: d.lower() if pd.isna(d) == False else d)
df["designation_tr"] = df["designation_tr"].replace(["reserva","riserva","rèserve"],["reserve","reserve","reserve"])


df

# rimuovo regioni doppie e creo regione unica
df.loc[df["region_1"] == df["region_2"], "region_2"] = ""
df["region_complete"] = df[["region_1","region_2"]].fillna(" ").agg(' '.join, axis=1)

singular_region_complete = set(df["region_complete"])
len(singular_region_complete)

import geocoder

g = geocoder.arcgis('Redlands, CA')
print(g.latlng)

import geocoder

singular_region_complete_with_latlong = []
for region in singular_region_complete:
    print(region)
    loc =  geocoder.arcgis(region,maxRows=1)
    print(loc)
    try:
        singular_region_complete_with_latlong.append((region, loc.latlng[0],loc.latlng[1]))   
    except:
        pass

with open('region_coordinates.csv', 'w') as fp:
    fp.write('\n'.join('%s,%f,%f' % x for x in singular_region_complete_with_latlong))

df_region_coordinates = pd.read_csv('region_coordinates.csv')

df = pd.merge(df, df_region_coordinates, how="left", on="region_complete")

df.to_csv("competition_dataset/mine_dev.csv",index=False)

df = pd.get_dummies(df, columns=["designation_tr","province","variety"])

df.columns

df = pd.read_csv("competition_dataset/mine_dev.csv")

with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    df["designation_tr"].value_counts()[:200]
    
len(df["designation_tr"].value_counts())

import gmaps
import gmaps.datasets
from ipywidgets.embed import embed_minimal_html

locations = df[['lat', 'long']]
weights = df['quality']
fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(locations, weights=weights)
fig.add_layer(heatmap_layer)
embed_minimal_html('export.html', views=[fig])

df["variety"].value_counts()[:50]

df["province"].value_counts()[:50]

df["region_complete"] = df["region_complete"].replace(r'^\s*$', np.NaN, regex=True)


df = pd.get_dummies(df, columns=["region_complete","variety"])

df.to_csv("competition_dataset/mine_dev_with_dummies.csv",index=False)

df = pd.read_csv("competition_dataset/mine_dev_with_dummies.csv")

len(df.columns)

from platform import python_version

print(python_version())


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
#Caricamento di x
df = pd.read_csv("competition_dataset/dev.tsv", sep="\t")

In [ ]:
df.info()

df_test = df.drop_duplicates()

df_test[df_test["province"] == "Idaho"]

df_test.info()

mask = (df_test['region_1'].isnull()) & (df_test['region_2'].isnull()) & df_test['province'].isnull()

df_test[mask].info()

# In 13889 record entrambe le region sono nulle
# In 3 record entrambe le region sono nulle e anche la province

mask = df_test['region_1'].isnull() & df_test['region_2'].isnull()

province_with_region_nans = df_test[mask]["province"].unique()

print(len(province_with_region_nans))

df_test[df_test["province"] == "Mendoza Province"]

mask_3 = (df_test['region_1'].isnull()) & (df_test['region_2'].isnull())
df_test[mask_3]

# In 13889 record entrambe le region sono nulle

mask_2 = (df_test['region_1'].isnull() & df_test['region_2'].isnull()) == False

df_test_at_least_one_region = df_test[mask_2]

province_with_at_least_one_region = df_test[mask_2]["province"].unique()

df_test_at_least_one_region[df_test_at_least_one_region["province"].isin(province_with_region_nans)]

country_with_at_least_one_region = df_test[mask_2]["country"].unique()

province_with_at_least_one_region

df_test = df.drop_duplicates()

mask_2 = (df_test['region_1'].isnull() & df_test['region_2'].isnull()) == False
province_with_at_least_one_region = df_test[mask_2]["province"].unique()

mask4 = (df_test['region_1'].isnull()) & (df_test['region_2'].isnull()) & (df_test['province'].isin(province_with_at_least_one_region))
df_test_no_region_in_at_least_one_prov = df_test[mask4].copy() # province che non hanno region in almeno un record e hanno region in almeno un record

province_interested = df_test_no_region_in_at_least_one_prov["province"].unique()

# In 88 record le due regioni sono nulle e la provincia ha almeno un record con una regione

mask = df_test["province"].isin(province_interested)
df_test_interested = df_test[mask].copy()

df_test_interested.loc[df_test_interested["region_1"] == df_test_interested["region_2"], "region_2"] = "" # se region1 e 2 sono uguali rendo region 2 una stringa vuota
df_test_interested["region_complete"] = df_test_interested[["region_1","region_2"]].fillna(" ").agg(' '.join, axis=1)
df_test_interested["region_complete"] = df_test_interested["region_complete"].replace(r'^\s*$', np.NaN, regex=True)

provinces = (df_test_interested[["province","region_complete"]].groupby(['province']).agg(lambda x:x.value_counts().index[0] if len(x.value_counts().index) > 0 else "Null").to_dict())

df_test_interested[((df_test_interested["region_1"].isnull() == True) & (df_test_interested["region_2"].isnull() == True))].info()

mask6 = (df_test['region_1'].isnull()) & (df_test['region_2'].isnull()) & (df_test['country'].isin(country_with_at_least_one_region))
df_test_no_region_in_at_least_one_coun = df_test[mask6]

df_test_no_region_in_at_least_one_prov.info()

# In 88 record le due regioni sono nulle e la nazione ha almeno un record con una regione

mask5 = (df_test['region_1'].isnull()) & (df_test['region_2'].isnull()) & (df_test['province'].isnull())

df_test_no_region_no_prov = df_test[mask5]

df_test_no_region_no_prov.info()

In [ ]:
import pandas as pd
import numpy as np
import math

In [ ]:
#Caricamento di x
df = pd.read_csv("competition_dataset/dev.tsv", sep="\t")

In [ ]:
import matplotlib.pyplot as plt

myFig = plt.figure();
ax, bp = df.boxplot(column=['quality'], return_type="both")
ax.set_title("Boxplot of quality")
myFig.savefig("myName.svg", format="svg")

outliers = [flier.get_ydata() for flier in bp["fliers"]]
boxes = [box.get_ydata() for box in bp["boxes"]]
medians = [median.get_ydata() for median in bp["medians"]]
whiskers = [whiskers.get_ydata() for whiskers in bp["whiskers"]]
print(outliers, boxes, medians, whiskers)

In [ ]:
quantiles = df.quantile([0.01, 0.25, 0.5, 0.75, 0.99])
quantiles

In [ ]:
len(df)-len(df.drop_duplicates())

In [ ]:
(df[df["quality"] ==0])

In [ ]:
df_test = df.drop_duplicates()

df_test[df_test["region_1"] == df_test["region_2"]]

In [ ]:
myFig = plt.figure();
ax = df_test["designation"].value_counts()[:20][::-1].plot(kind='barh')
ax.set_title("Top 20 designations in the dataset")
myFig.savefig("Top_20_designations.svg", format="svg",bbox_inches='tight')


In [ ]:
outliers = [flier.get_ydata() for flier in bp["fliers"]]
sorted(outliers)

In [ ]:
df[df["country"].isna()]

In [ ]:
df.info()

In [ ]:
# Preprocessing su x
print(len(df))
df = df.drop_duplicates() # Rimuovo duplicati
print(len(df))


df = df.drop(df[df["quality"] == 0].index) #Rimuovo righe con qualità uguale a zero ( sono solo 15 in totale )
df.reset_index(drop=True, inplace=True)

df["designation_tr"] = df["designation"].apply(lambda d: d.lower() if pd.isna(d) == False else d)
df["designation_tr"] = df["designation_tr"].replace(["reserva","riserva","réserve"],["reserve","reserve","reserve"])


df.loc[df["region_1"] == df["region_2"], "region_2"] = "" # se region1 e 2 sono uguali rendo region 2 una stringa vuota
df["region_complete"] = df[["region_1","region_2"]].fillna(" ").agg(' '.join, axis=1)
df["region_complete"] = df["region_complete"].replace(r'^\s*$', np.NaN, regex=True)

df["description_len"] = df["description"].map(len)

# --------------------------------------------
# Riempio alcune region_complete nulle


mask_1 = (df['region_1'].isnull() & df['region_2'].isnull()) == False
province_with_at_least_one_region = df[mask_1]["province"].unique()

mask_2 = (df['region_1'].isnull()) & (df['region_2'].isnull()) & (df['province'].isin(province_with_at_least_one_region))
df_no_region_in_at_least_one_prov = df[mask_2].copy() # province che non hanno region in almeno un record e hanno region in almeno un record

province_interested = df_no_region_in_at_least_one_prov["province"].unique()

# In 88 record le due regioni sono nulle e la provincia ha almeno un record con una regione

mask = df["province"].isin(province_interested)
df_interested = df[mask].copy()

province_region = (df_interested[["province","region_complete"]].groupby(['province']).agg(lambda x:x.value_counts().index[0] if len(x.value_counts().index) > 0 else "Null").to_dict())

province_region = province_region["region_complete"]

def change_region_complete(x):    
    return province_region[x["province"]] if (type(x["region_complete"]) == float and math.isnan(x["region_complete"])) else x["region_complete"]

df.loc[mask, "region_complete"] = df.loc[mask].apply(change_region_complete, axis=1) 

# --------------------------------------------

df["region_complete"].fillna('None', inplace=True)
df["variety"].fillna('None', inplace=True)
df["country"].fillna('None', inplace=True)
df["winery"].fillna('None', inplace=True)
df["province"].fillna('None', inplace=True)
df["designation_tr"].fillna('None', inplace=True)

df.plot.scatter(x='description_len', y='quality')

import matplotlib.pyplot as plt

sample = df.sample(n=1000)
sample.plot.scatter(x='description_len', y='quality', alpha=0.7)
plt.show()


In [ ]:
df["designation_tr"].value_counts()

In [ ]:
# Caricamento di x_eval

df_eval = pd.read_csv("competition_dataset/eval.tsv", sep="\t")

In [ ]:
df_eval.info()

In [ ]:
# Preprocessing su x_eval

df_eval["designation_tr"] = df_eval["designation"].apply(lambda d: d.lower() if pd.isna(d) == False else d)
df_eval["designation_tr"] = df_eval["designation_tr"].replace(["reserva","riserva","réserve"],["reserve","reserve","reserve"])

df_eval.loc[df_eval["region_1"] == df_eval["region_2"], "region_2"] = ""
df_eval["region_complete"] = df_eval[["region_1","region_2"]].fillna(" ").agg(' '.join, axis=1)
df_eval["region_complete"] = df_eval["region_complete"].replace(r'^\s*$', np.NaN, regex=True)

df_eval["description_len"] = df_eval["description"].map(len)

mask = df_eval["province"].isin(province_interested)
df_eval.loc[mask, "region_complete"] = df_eval.loc[mask].apply(change_region_complete, axis=1) 


df_eval["region_complete"].fillna('None', inplace=True)
df_eval["variety"].fillna('None', inplace=True)
df_eval["country"].fillna('None', inplace=True)
df_eval["winery"].fillna('None', inplace=True)
df_eval["province"].fillna('None', inplace=True)
df_eval["designation_tr"].fillna('None', inplace=True)

In [ ]:
#Aggiungo a x i bit delle winery con più vini

from sklearn.preprocessing import OneHotEncoder

df_group_by_winery = df[["winery","quality"]].groupby(by="winery",sort=True).agg(['mean', 'count']).sort_values(by=("quality","count"),ascending=False)


top_N_winery = df_group_by_winery[df_group_by_winery[("quality","count")] >= 20].index.values

df.loc[~df["winery"].isin(top_N_winery) ,"winery"] = "None"

ohc_w = OneHotEncoder(handle_unknown="ignore")

ohWinery = ohc_w.fit_transform(df["winery"].values.reshape(-1,1)).toarray()
dfOneHot_w = pd.DataFrame(ohWinery, columns=["winery_" + str(ohc_w.categories_[0][i]) for i in range(len(ohc_w.categories_[0]))])

df = pd.concat([df,dfOneHot_w], axis=1)


In [ ]:
top_N_winery

In [ ]:
df["province"].value_counts()

In [ ]:
#Aggiungo a x_eval i bit delle winery con più vini

from sklearn.preprocessing import OneHotEncoder

ohWinery = ohc_w.transform(df_eval["winery"].values.reshape(-1,1)).toarray()
dfOneHot_w = pd.DataFrame(ohWinery, columns=["winery_" + str(ohc_w.categories_[0][i]) for i in range(len(ohc_w.categories_[0]))])

df_eval = pd.concat([df_eval,dfOneHot_w], axis=1)


In [ ]:
df

In [ ]:
df[df["quality"] == 7]

#Aggiungo coordinate lat long a x
df_region_coordinates = pd.read_csv('region_coordinates.csv')

df = pd.merge(df, df_region_coordinates, how="left", on="region_complete")

print(df.columns)
lat_mean = df['lat'].mean()
long_mean = df['long'].mean()

df["lat"].fillna(lat_mean, inplace=True)
df["long"].fillna(long_mean, inplace=True)


#Aggiungo coordinate lat long a x_eval
df_region_coordinates = pd.read_csv('region_coordinates.csv')

df_eval = pd.merge(df_eval, df_region_coordinates, how="left", on="region_complete")

In [ ]:
from scipy.sparse import coo_matrix, hstack

# Aggiungo a x bit di encoding di region_complete e variety e country e province
from sklearn.preprocessing import OneHotEncoder

ohc_r = OneHotEncoder(handle_unknown="ignore")
ohc_v = OneHotEncoder(handle_unknown="ignore")
ohc_c = OneHotEncoder(handle_unknown="ignore")
ohc_p = OneHotEncoder(handle_unknown="ignore")

dfOneHot_r = pd.DataFrame(ohRegion, columns=["region_complete_" + str(ohc_r.categories_[0][i]) for i in range(len(ohc_r.categories_[0]))])
dfOneHot_v = pd.DataFrame(ohVariety, columns=["variety_" + str(ohc_v.categories_[0][i]) for i in range(len(ohc_v.categories_[0]))])
dfOneHot_c = pd.DataFrame(ohCountry, columns=["country_" + str(ohc_c.categories_[0][i]) for i in range(len(ohc_c.categories_[0]))])
dfOneHot_p = pd.DataFrame(ohProvince, columns=["province_" + str(ohc_p.categories_[0][i]) for i in range(len(ohc_p.categories_[0]))])

df = pd.concat([df,dfOneHot_r,dfOneHot_v, dfOneHot_c, dfOneHot_p], axis=1)

In [ ]:
# Aggiungo a x_eval bit di encoding di region_complete e variety e country e province

dfOneHot_r = pd.DataFrame(ohRegion, columns=["region_complete_" + str(ohc_r.categories_[0][i]) for i in range(len(ohc_r.categories_[0]))])
dfOneHot_v = pd.DataFrame(ohVariety, columns=["variety_" + str(ohc_v.categories_[0][i]) for i in range(len(ohc_v.categories_[0]))])
dfOneHot_c = pd.DataFrame(ohCountry, columns=["country_" + str(ohc_c.categories_[0][i]) for i in range(len(ohc_c.categories_[0]))])
dfOneHot_p = pd.DataFrame(ohProvince, columns=["province_" + str(ohc_p.categories_[0][i]) for i in range(len(ohc_p.categories_[0]))])

df_eval = pd.concat([df_eval,dfOneHot_r,dfOneHot_v,dfOneHot_c,dfOneHot_p], axis=1)


df_group_by_designation_tr = df[["designation_tr","quality"]].groupby(by="designation_tr",sort=True).agg(['mean','count'])
df_group_by_designation_tr.reset_index()
df_group_by_designation_tr = df_group_by_designation_tr[df_group_by_designation_tr[("quality","count")] >= 10]

df_group_by_designation_tr.drop(["None"], inplace=True)

df_group_by_designation_tr.columns = df_group_by_designation_tr.columns.droplevel(0)
df_group_by_designation_tr.drop(columns=["count"], inplace=True)
df_group_by_designation_tr.columns = ["designation_tr_quality_mean"]

df_group_by_designation_tr
#df = pd.merge(df, df_group_by_designation_tr, how="left", on="designation_tr")

df_test = pd.merge(df, df_group_by_designation_tr, how="left", on="designation_tr")

df_test["designation_tr_quality_mean"].fillna(-1, inplace=True)

# Aggiungo a x i chunks delle designation
from sklearn.preprocessing import OneHotEncoder

df_group_by_designation_tr = df[["designation_tr","quality"]].groupby(by="designation_tr",sort=True).agg(['mean','count'])
df_group_by_designation_tr.reset_index()
df_group_by_designation_tr = df_group_by_designation_tr[df_group_by_designation_tr[("quality","count")] >= 10]

df_group_by_designation_tr.drop(["None"], inplace=True)

df_group_by_designation_tr.columns = df_group_by_designation_tr.columns.droplevel(0)
df_group_by_designation_tr.drop(columns=["count"], inplace=True)
df_group_by_designation_tr.columns = ["desi_quality_mean"]


df = pd.merge(df, df_group_by_designation_tr, how="left", on="designation_tr")
df["desi_quality_mean"].fillna(-1, inplace=True)
df["desi_quality_mean"] = df["desi_quality_mean"].astype(int)

ohc_d = OneHotEncoder(handle_unknown="ignore")
ohDesignation_quality_mean = ohc_d.fit_transform(df["desi_quality_mean"].values.reshape(-1,1)).toarray()

dfOneHot_dq = pd.DataFrame(ohDesignation_quality_mean, columns=["desi_quality_mean_" + str(ohc_d.categories_[0][i]) for i in range(len(ohc_d.categories_[0]))])

df.drop(columns=["desi_quality_mean"], inplace=True)
 
df = pd.concat([df,dfOneHot_dq], axis=1)

# Aggiungo a x_eval i chunks delle designation
from sklearn.preprocessing import OneHotEncoder

df_eval = pd.merge(df_eval, df_group_by_designation_tr, how="left", on="designation_tr")

df_eval["desi_quality_mean"] = df_eval["desi_quality_mean"].fillna(-1)

ohDesignation_quality_mean = ohc_d.transform(df_eval["desi_quality_mean"].values.reshape(-1,1)).toarray()

dfOneHot_dq = pd.DataFrame(ohDesignation_quality_mean, columns=["desi_quality_mean_" + str(ohc_d.categories_[0][i]) for i in range(len(ohc_d.categories_[0]))])

df_eval.drop(columns=["desi_quality_mean"], inplace=True)

df_eval = pd.concat([df_eval,dfOneHot_dq], axis=1)

In [ ]:
#Aggiungo i bit dei primi N designation a x
from sklearn.preprocessing import OneHotEncoder

N = 300 
top_N_designation_tr = list(df["designation_tr"].value_counts()[1:N+1].keys()) #non includo None values

df.loc[~df["designation_tr"].isin(top_N_designation_tr) ,"designation_tr"] = "None"

ohc_d = OneHotEncoder(handle_unknown="ignore") 
ohDesignation_tr = ohc_d.fit_transform(df["designation_tr"].values.reshape(-1,1)).toarray()

dfOneHot_d = pd.DataFrame(ohDesignation_tr, columns=["designationtr" + str(ohcd.categories[0][i]) for i in range(len(ohcd.categories[0]))])

df = pd.concat([df,dfOneHot_d], axis=1)



In [ ]:
#Aggiungo i bit dei primi N designation a x eval
from sklearn.preprocessing import OneHotEncoder

ohDesignation = ohc_d.transform(df_eval["designation_tr"].values.reshape(-1,1)).toarray() 
dfOneHot_d = pd.DataFrame(ohDesignation, columns=["designationtr" + str(ohcd.categories[0][i]) for i in range(len(ohcd.categories[0]))])

df_eval = pd.concat([df_eval,dfOneHot_d], axis=1)

In [ ]:
#Aggiungo a x i bit delle N parole più frequenti nella description 
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix

vectorizer = TfidfVectorizer(stop_words="english", binary=True, use_idf=False,norm=False)

wpm = vectorizer.fit_transform(df["description"].fillna(""))

N = 1000
freq = sorted(zip(vectorizer.get_feature_names(), wpm.sum(axis=0).tolist()[0]),key=lambda x: x[1], reverse=True)[:N]

words = [ word for word, _ in freq ]
mask = [ w in words for w in vectorizer.get_feature_names() ]
words_ = [ w for w in vectorizer.get_feature_names() if w in words ]
df_words = pd.DataFrame(data=wpm[:, np.array(mask)].toarray(),columns=[f"word_{word}" for word in words_], index=df.index)

df = pd.concat([df,df_words], axis=1)

In [ ]:
vectorizer.get_feature_names()

In [ ]:
#Aggiungo a x_eval i bit delle N parole più frequenti nella description 
from sklearn.feature_extraction.text import TfidfVectorizer

wpm = vectorizer.transform(df_eval["description"].fillna(""))

df_words = pd.DataFrame(data=wpm[:, np.array(mask)].toarray(),columns=[f"word_{word}" for word in words_], index=df_eval.index)

df_eval = pd.concat([df_eval,df_words_eval], axis=1)

# TEST Aggiungo a x i bit delle N parole più frequenti nella description 
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words="english", binary=False, use_idf=False)

wpm = vectorizer.fit_transform(df["description"].fillna(""))

N = 1000
freq = sorted(zip(vectorizer.get_feature_names(), wpm.sum(axis=0).tolist()[0]),key=lambda x: x[1], reverse=True)[:N]

words = [ word for word, _ in freq ]
mask = [ w in words for w in vectorizer.get_feature_names() ]
words_ = [ w for w in vectorizer.get_feature_names() if w in words ]
df_words = pd.DataFrame(data=wpm[:, np.array(mask)].toarray(),columns=[f"word_{word}" for word in words_], index=df.index)

df = pd.concat([df,df_words], axis=1)
df_words

df_words[df_words['word_000'].gt(0)].index[0]

df_words.loc[74, "word_000"]

# TEST Aggiungo a x_eval i bit delle N parole più frequenti nella description 
from sklearn.feature_extraction.text import TfidfVectorizer

wpm = vectorizer.transform(df_eval["description"].fillna(""))

df_words_eval = pd.DataFrame(data=wpm[:, np.array(mask)].toarray(),columns=[f"word_{word}" for word in words_], index=df_eval.index)

df_eval = pd.concat([df_eval,df_words_eval], axis=1)


#Test tdidf description su x
from sklearn.feature_extraction.text import TfidfVectorizer

N = 500
vectorizer = TfidfVectorizer(stop_words="english", use_idf=True, max_features = N)

wpm = vectorizer.fit_transform(df["description"].fillna(""))

words = [ word for word, _ in freq ]
mask = [ w in words for w in vectorizer.get_feature_names() ]
words_ = [ w for w in vectorizer.get_feature_names() if w in words ]
df_words = pd.DataFrame(data=wpm[:, np.array(mask)].toarray(),columns=[f"word_{word}" for word in words_], index=df.index)

df_words

In [ ]:
from sklearn.ensemble import RandomForestRegressor

regr = RandomForestRegressor(n_estimators=100, n_jobs=-1,max_features="sqrt")

In [ ]:
df.iloc[:,9:]

#Test PCA 
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler


from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

X = df.iloc[:, 11:]
y = df["quality"]

scaler = MinMaxScaler()
X_rescaled = scaler.fit_transform(X)
#y_rescaled = scaler.fit_transform(y)

pca = PCA(n_components = 0.99)
pca.fit(X_rescaled)
reduced = pca.transform(X_rescaled)

reduced


reduced.shape

#Predict con PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
            
X_train, X_test, y_train, y_test = train_test_split(reduced, y, test_size=0.25, random_state=34)

regr.fit(X_train,y_train)
            
y_pred = regr.predict(X_test)

r2_score(y_test, y_pred)

In [ ]:
X.shape

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

X = df.iloc[:, 11:]
y = df["quality"]
            
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=34)

regr.fit(X_train,y_train)
            
y_pred = regr.predict(X_test)

r2_score(y_test, y_pred)

In [ ]:
# togliendo i duplicati r2 score locale  0.5355624548339075

In [ ]:
# features importances
import collections

feat_importances = sorted(zip(list(X.columns[11:]), regr.feature_importances_), key=lambda x: x[1],reverse=True)[:100]

features = [element[0] for element in feat_importances]
features = [element.split("_")[0] for element in features]

collections.Counter(features)


In [ ]:
#Ricerca con grid search

from sklearn.model_selection import GridSearchCV

X = df.iloc[:, 11:]
y = df["quality"]
            
param_grid = {
"n_estimators": [100, 250],
"criterion": ["mse"],
"max_features": ["auto", "sqrt", "log2"],
"random_state": [42], # always use the samet random seed
"n_jobs": [-1], # for parallelization
}

gs = GridSearchCV(RandomForestRegressor(), param_grid, scoring="r2", n_jobs=-1,cv=5)
gs.fit(X, y)
gs.best_score_

In [ ]:
from prettytable import PrettyTable

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.preprocessing import FunctionTransformer, PolynomialFeatures
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline

def evaluate_model(X, y, model, model_name):

    X_train, X_test, y_train, y_test = train_test_split(X, y,train_size=0.25,random_state=42,shuffle=True)
    # plot the real function and the training points
    LW = 2
    #fig, ax = plt.subplots()
    #ax.plot(X, y, color='cornflowerblue', linewidth=.5*LW, label="ground truth")
    #ax.scatter(X_train, y_train, color='navy', s=30, marker='o',label="training points")
    # predict the test points and plot them onto the chart
    model.fit(X_train, y_train)
    y_hat = model.predict(X_test)
    #ax.plot(X_test, y_hat, linewidth=LW, label=name, color='r')
    #fig.suptitle(f"{f} approximated by {model_name}")
    #fig.legend()
    return r2_score(y_test, y_hat)



degree = 2
models = [
    LinearRegression(),
    Ridge(random_state=42),
    RandomForestRegressor(n_estimators=100,n_jobs=-1,max_features="sqrt"),
    make_pipeline(
        make_column_transformer(
            (FunctionTransformer(np.sin), [0]),
            (PolynomialFeatures(degree), [0])
        ),
        LinearRegression()
    ),
    make_pipeline(
        make_column_transformer(
            (FunctionTransformer(np.sin), [0]),
            (PolynomialFeatures(degree), [0])
        ),
        Ridge(alpha=1)
    )
]

names = [
    'linreg',
    'ridge',
    'rf',
    f'sin+poly{degree}+linreg',
    f'sin+poly{degree}+ridge'
]


t = PrettyTable()
t.field_names = ['model', 'R2']
X = df.iloc[:, 11:]
y = df["quality"]
for model, name in zip(models, names):
    print(model, name)
    r2 = evaluate_model(X, y, model, name)
    t.add_row([name, r2])
print(t)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

X = df.iloc[:, 11:]
y = df["quality"]

regr_rf = RandomForestRegressor()
parameters_rf = {'n_estimators':[50,100], 'n_jobs':[-1], "max_features":["sqrt","log2"], "random_state":[34]}
clf_rf = GridSearchCV(regr_rf, parameters_rf, scoring="r2",verbose=3)

clf_rf.fit(X,y)
print(clf_rf.best_params_)
print(clf_rf.best_score_)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge

X = df.iloc[:, 11:]
y = df["quality"]

regr_rf = Ridge()
parameters_rf = { "alpha":np.arange(0.1, 1.1, 0.2), "fit_intercept":[True,False],"random_state":[34]}
clf_rf = GridSearchCV(regr_rf, parameters_rf, scoring="r2",verbose=3)

clf_rf.fit(X,y)
print(clf_rf.best_params_)
print(clf_rf.best_score_)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso

X = df.iloc[:, 11:]
y = df["quality"]

regr_rf = Lasso()
parameters_rf = { "alpha":np.arange(0.1, 1.1, 0.2), "fit_intercept":[True,False],"random_state":[34]}
clf_rf = GridSearchCV(regr_rf, parameters_rf, scoring="r2",verbose=3)

clf_rf.fit(X,y)
print(clf_rf.best_params_)
print(clf_rf.best_score_)

In [ ]:
freq[:100]

In [ ]:
X.shape

In [ ]:
sorted(zip(list(X.columns[11:]), regr.feature_importances_), key=lambda x: x[1],reverse=True)

In [ ]:
X_train["lat"].isnull().sum()

In [ ]:
list(set(list(df_eval.iloc[:,10:].columns)) - set(list(df.iloc[:, 11:].columns)))

In [ ]:
df_eval.drop(columns=["desi_quality_mean"], inplace=True)


In [ ]:
df_eval.iloc[:,10:]

In [ ]:
df.iloc[:, 11:]

In [ ]:
from sklearn.ensemble import RandomForestRegressor

regr = RandomForestRegressor(n_estimators=100, n_jobs=-1,max_features="sqrt")

In [ ]:
X = df.iloc[:, 11:]
y = df["quality"]
            
regr.fit(X,y)

In [ ]:
X_eval = df_eval.iloc[:,10:]

y_eval = regr.predict(X_eval)

In [ ]:
y_eval

In [ ]:
with open("my_submission.csv","w") as f:
    f.write("Id,Predicted\n")
    for i,result in enumerate(y_eval):
        f.write(str(i) + "," + str(y_eval[i])+"\n")